In [62]:
import requests
import random
import time
import os
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
# from scrape_functions import (
#     get_comic_title,
#     find_image,
#     scrape_image,
#     get_prev)

from selenium.webdriver.firefox.options import Options

options = Options()
options.headless = True

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs
import os

# Custom
import data_fcns as dfc
import keys  # Custom keys lib

import data_fcns as dfc

import pandas as pd

## Initialize Stuff

In [63]:
browser = Firefox(options=options)
url = "http://www.comicbookdb.com/"
browser.get(url)

## Make list of Titles!

In [5]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

'/Users/werlindo/.secret/aws_ps_flatiron.json'

In [6]:
aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [7]:
# Instantiate cursor
cur = conn.cursor()

In [8]:
#  Count records.
query = """
    SELECT * from comic_trans;
"""

In [9]:
# Execute the query
cur.execute(query)

In [10]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

In [11]:
temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG)
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG)
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG)


In [12]:
temp_df['title'] = ( temp_df['title_and_num'].apply(dfc.cut_issue_num) )

In [73]:
temp_df.head()

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,title
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),Filler Bunny
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),Gargoyles
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),Royal Historian of Oz
3,5,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-14 19:49:40,00399,Royal Historian of Oz (SLG),Royal Historian of Oz
4,6,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-19 10:39:04,00237,Royal Historian of Oz (SLG),Royal Historian of Oz


In [14]:
temp_df['title'] = temp_df['title'].apply(lambda x : x.replace('&' ,'and'))

In [15]:
titles = list(temp_df['title'].unique())

In [51]:
99+189

288

In [66]:
sample_titles = titles[:300]

In [67]:
sample_titles

['Cursed Pirate Girl Special An', 'Dadoes Dust To Dust', 'Darkwing Duck']

Get list, sorted by qty sold

In [92]:
qtys = temp_df.groupby(['title'], as_index=False).qty_sold.sum().sort_values(by=['qty_sold'], ascending=False)

In [94]:
qtys.head()

,title,qty_sold
553,Batman,9562
6700,Walking Dead,6856
229,Amazing Spider-Man,5828
5098,Saga,5542
5929,Superman,5197


In [99]:
done_titles = titles[:300]

In [103]:
titles_needed_df = qtys.loc[~qtys['title'].isin(done_titles)]

In [104]:
titles_needed_df.shape

(6773, 2)

In [106]:
titles_need_list = list(titles_needed_df.title.unique())

In [107]:
titles_need_list[:10]

['Batman',
 'Walking Dead',
 'Amazing Spider-Man',
 'Saga',
 'Superman',
 'Justice League',
 'Detective Comics',
 'Uncanny X-Men',
 'Wonder Woman',
 'Star Wars']

In [112]:
titles_searching = titles_need_list[11:]

In [113]:
titles_searching

['Flash',
 'Avengers',
 'New Avengers',
 'Daredevil',
 'East of West',
 'Deadpool',
 'Action Comics',
 'Batman and Robin',
 'Batgirl',
 'Green Lantern',
 'All New X-Men',
 'Sixth Gun',
 'Chew',
 'Captain America',
 'Uncanny X-Force',
 'Invincible Iron Man',
 'Batwoman',
 'Invincible',
 'Fantastic Four',
 'Wolverine and X-Men',
 'Superman Unchained',
 'Justice League of America',
 'Mighty Thor',
 'Uncanny Avengers',
 'Aquaman',
 'Green Arrow',
 'Descender',
 'Outcast By Kirkman and Azaceta',
 'Nightwing',
 'Batman Eternal',
 'Fatale',
 'Fables',
 'Green Lantern Corps',
 'Doctor Strange',
 'Guardians of Galaxy',
 'Darth Vader',
 'Superior Spider-Man',
 'Sweet Tooth',
 'Brightest Day',
 'Ms Marvel',
 'Zombie Tramp Ongoing',
 'Lazarus',
 'Manhattan Projects',
 'Animal Man',
 'Teen Titans',
 'Paper Girls',
 'Avengers (2010)',
 'Astonishing X-Men',
 'Invader Zim',
 'Black Science',
 'Wicked and Divine',
 'Secret Empire',
 'Sex Criminals',
 'Old Man Logan',
 'Swamp Thing',
 'Supergirl',
 'X-M

## It's the Scraping.

In [114]:
# for title in sample_titles:
# #     print(title)
scrape_series_covers(browser, titles_searching)

Scraped 0.Flash!
Scraped 1.Avengers!
Scraped 2.New Avengers!
Scraped 3.Daredevil!
Scraped 4.East of West!
Scraped 5.Deadpool!
Scraped 6.Action Comics!
Scraped 7.Batman and Robin!
Scraped 8.Batgirl!
Scraped 9.Green Lantern!
Scraped 10.All New X-Men!
Scraped 11.Sixth Gun!
Scraped 12.Chew!
Scraped 13.Captain America!
Scraped 14.Uncanny X-Force!
Scraped 15.Invincible Iron Man!
Scraped 16.Batwoman!
Scraped 17.Invincible!
Scraped 18.Fantastic Four!
Scraped 19.Wolverine and X-Men!
Scraped 20.Superman Unchained!
Scraped 21.Justice League of America!
Scraped 22.Mighty Thor!
Scraped 23.Uncanny Avengers!
Scraped 24.Aquaman!
Scraped 25.Green Arrow!
Scraped 26.Descender!
27.Outcast By Kirkman and Azaceta was skipped. No title matched.
Scraped 28.Nightwing!
Scraped 29.Batman Eternal!
Scraped 30.Fatale!
Scraped 31.Fables!
Scraped 32.Green Lantern Corps!
Scraped 33.Doctor Strange!
Scraped 34.Guardians of Galaxy!
Scraped 35.Darth Vader!
Scraped 36.Superior Spider-Man!
Scraped 37.Sweet Tooth!
Scraped 38

Scraped 279.War of Realms!
Scraped 280.Demon Knights!
Scraped 281.Goon!
Scraped 282.Rebels!
Scraped 283.American Vampire Second Cycl!
Scraped 284.Captain America and Bucky!
Scraped 285.We Stand On Guard!
Scraped 286.House of Whispers!
Scraped 287.Moon Girl and Devil Dinosaur!
Scraped 288.Stormwatch!
Scraped 289.Aliens Dead Orbit!
Scraped 290.Batman Streets of Gotham!
Scraped 291.Sex!
Scraped 292.Justice Society of America!
Scraped 293.Blue Estate!
Scraped 294.Batman Who Laughs!
295.Neil Gaiman American Gods My was skipped. No title matched.
Scraped 296.All New X-Factor!
Scraped 297.Serenity Leaves On the Wind!
Scraped 298.Robin Son of Batman!
Scraped 299.Booster Gold!
Scraped 300.Grayson!
Scraped 301.Inhuman!
Scraped 302.Hit-Girl!
Scraped 303.X-Force!
Scraped 304.Secret!
Scraped 305.Zatanna!
Scraped 306.Black Hammer Age of Doom!
Scraped 307.Flashpoint!
Scraped 308.Satellite Sam!
Scraped 309.Jsa Liberty Files the Whistli!
Scraped 310.Constantine!
Scraped 311.Princess Leia!
Scraped 312.S

KeyboardInterrupt: 

## Master Function Testing

In [72]:
def scrape_series_covers(browser, titles):
    """Use Selenium to scrape images for comic book titles"""
    
    start_time = time.time()
    
    for idx, title in enumerate(titles):
        # Search for the title
        search_title(browser, title)

#         title_search_flag = not no_results_found()

#         # Check if no results found
#         if not title_search_flag:
#             search_site(browser, title)
        
        if not no_results_found():
        # Once on search results, just select first issue of results
            click_first_link(browser, title, True)

            # Go to the cover gallery of issue page
            go_cover_gallery(browser)

            # Once in cover gallery, just scrape the first image
            try:
                get_first_image(browser, title)
                print("Scraped {}.{}!".format(idx, title))
            except NoSuchElementException:
                print("{}.{} was skipped. No covers were found."
                      .format(idx, title))

                # Go back to homepage so can do it again!
                #go_back_home_comicbookdb(browser)
        else:
            print("{}.{} was skipped. No title matched.".format(idx, title))
            
            
            # Wait random time
            time.sleep(2 + random.random()*5)
    
    print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

#     print("All done!")

In [29]:
def no_results_found():
    xpath = '/html/body/table/tbody/tr[2]/td[3]'
    result = browser.find_element_by_xpath(xpath)
    return result.text == 'No results found.'

## Search on Title

In [17]:
def search_title(browser, title):
    """
    Given Selenium browser obj and a comic title to search for
    Enter title into search box and Search
    """
    # Find search box and enter search text
    text_area = browser.find_element_by_id('form_search')
    text_area.send_keys(Keys.CONTROL, "a") 
    text_area.send_keys(title)
    
    # Find Search type dropdown and make sure it says 'Title'
    search_type = Select(browser.find_element_by_name('form_searchtype'))
    search_type.select_by_value('Title')

    
    # Push the search button!
    sb_xpath = ('/html/body/table/tbody/tr[2]/td[1]' + 
                '/table/tbody/tr[4]/td/form/input[2]')
    search_button = browser.find_element_by_xpath(sb_xpath)
    search_button.click()


In [18]:
def search_site(browser, title):
    """
    Given Selenium browser obj and a comic title to search for
    Enter title into search box and Search
    """
    # Find search box and enter search text
    text_area = browser.find_element_by_id('form_search')
    text_area.send_keys(Keys.CONTROL, "a") 
    text_area.send_keys(title)
    
    # Find Search type dropdown and make sure it says 'Title'
#     search_type = Select(browser.find_element_by_name('form_searchtype'))
#     search_type.select_by_value('Title')
    
    # Push the search button!
    sb_xpath = ('/html/body/table/tbody/tr[2]/td[1]' + 
                '/table/tbody/tr[4]/td/form/input[2]')
    search_button = browser.find_element_by_xpath(sb_xpath)
    search_button.click()


In [42]:
title_search = "Filler Bunny"

In [61]:
title_search = 'Royal Historian of Oz'

In [68]:
title = title_search

In [62]:
text_area = browser.find_element_by_id('form_search')

In [63]:
text_area.send_keys(Keys.CONTROL, "a") 
text_area.send_keys(title_search)

In [64]:
search_type = Select(browser.find_element_by_name('form_searchtype'))

mySelect = Select(driver.find_element_by_id("mySelectID"))

In [65]:
search_type.select_by_value('Title')

In [66]:
search_button = browser.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[1]/table/tbody/tr[4]/td/form/input[2]')

In [67]:
search_button.click()

## Click on first result

In [19]:
def click_first_link(browser, title, title_search_flag):
    """
    Find first issue link and click it
    """
    # Find first issue link in search results
    if title_search_flag:
        x_path = '/html/body/table/tbody/tr[2]/td[3]/a[1]'
    else: 
        x_path = '/html/body/table/tbody/tr[2]/td[3]/table/tbody/tr/td/a[1]'
        
    #first_issue_link = browser.find_element_by_partial_link_text(title)
    first_issue_link = browser.find_element_by_xpath(x_path)
    
    # Click
    first_issue_link.click()


In [106]:
first_issue_link = browser.find_element_by_partial_link_text(title_search)

NoSuchElementException: Message: Unable to locate element: Royal Historian of Oz


In [70]:
first_issue_link.click()

## Go to Cover Gallery

In [20]:
def go_cover_gallery(browser):
    """
    Click on Cover Gallery button
    """
    gb_xpath = ("/html/body/table/tbody/tr[2]/td[3]/table[1]" + 
                "/tbody/tr/td/a[4]/img"
               )
    gb_xpath='//a[img/@src="graphics/button_title_covergallery.gif"]'
    
    gallery_btn = browser.find_element_by_xpath(gb_xpath)
    gallery_btn.click()

In [ ]:
/html/body/table/tbody/tr[2]/td[3]/table[1]/tbody/tr/td/a[4]/img

In [52]:
imgy = browser.fin

In [53]:
imgy.click()

In [ ]:
brow.find_element_by_xpath('//a[img/@src="resources/img/logout.png"]').click()

In [89]:
gb_xpath="/html/body/table/tbody/tr[2]/td[3]/table[1]/tbody/tr/td/a[3]/img"
gb_xpath="/html/body/table/tbody/tr[2]/td[3]/table[1]/tbody/tr/td/a[4]/img"
gb_xpath='//a[img/@src="graphics/button_title_covergallery.gif"]'
gallery_btn = browser.find_element_by_xpath(gb_xpath)

gallery_btn.click()

## Get First Image

In [32]:
def get_first_image(browser, title):
    """
    Find first image in cover gallery and scrape it!
    """
    # Find first image
    first_img_path = ('/html/body/table/tbody/tr[2]/td[3]/' + 
                      'table/tbody/tr[1]/td[1]/a/img')
    first_img = browser.find_element_by_xpath(first_img_path)    

    # Construct path and file name
    filename = ('./raw_data/covers/' + title.replace(' ', '_').lower() 
               + '.jpg'
               )
    
    # Save the file in the file/path 
    scrape_image(first_img, filename)
    
    return

In [72]:
first_img_path = '/html/body/table/tbody/tr[2]/td[3]/table/tbody/tr[1]/td[1]/a/img'

In [73]:
first_img = browser.find_element_by_xpath(first_img_path)

In [77]:
# Construct path and file name
filename = ('./assets/covers/' + title.replace(' ', '_').lower() 
           + '.jpg'
           )

In [22]:
def scrape_image(img, filename):
    """Save an image element as filename"""
    response = requests.get(img.get_attribute('src'))
    img_data = response.content
    with open(filename, 'wb') as f:
        f.write(img_data)

In [78]:
scrape_image(first_img, filename)

In [79]:
!ls

LICENSE
README.md
__pycache__
als_filtered
als_use
app.py
archive
assets
code_archive.py
comic_recs.py
comics_rx-1_data_prep.ipynb
comics_rx-2_eda.ipynb
comics_rx-3_als_all_data.ipynb
comics_rx-4_als_reduced_data.ipynb
comics_rx-5_pseudo_deploy.ipynb
comics_rx-6_als_reduced_data_gs_cv.ipynb
comics_rx-7_mvp_dev.ipynb
comics_rx-8_recommender_poc.ipynb
comics_rx-9_scrape_something.ipynb
data_fcns.py
dev
geckodriver.log
keys.py
raw_data
references
scrapes.py
scratch
spam_model.pkl
static
support_data
templates
testimage.jpg
versions


In [80]:
!open testimage.jpg

In [23]:
def go_back_home_comicbookdb(browser):
    # Find image link to go back home
    home_pg_xpath = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[1]/td/table/tbody/tr/td[1]/a/img'
    logo_btn = browser.find_element_by_xpath(home_pg_xpath)

    # Click!
    logo_btn.click()    
    

In [82]:
home_pg_xpath = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[1]/td/table/tbody/tr/td[1]/a/img'

logo_btn = browser.find_element_by_xpath(home_pg_xpath)

logo_btn.click()

---

# Create Loop for Titles

---

# Graveyard